# Working with Jobs
Unity helps teams move quickly from algorithm development and testing in Jupyter into large-scale processing with the Scaled Processing System (SPS). In order to do this, algorithms developed in Jupyter must be registered in the Application Catalog (see 1_working_with_applications.ipynb), deployed to an Application Deployment and Execution Service (ADES) in SPS, and then may be run on the SPS as _jobs_.

The intent of this tutorial is to help familiarize yourself with _jobs_. Unity provides users the ability to execute _jobs_ to produce data. The steps below will showcase how a job is typically submitted to the Unity Platform.

## 0. Set up imports and predefined variables

For this Tutorial we will make use of a wrapper API that is called api.py (in the utils folder). It helps to manage things like authentication.

In [ ]:
from utils import api
import json
import requests
from IPython.display import JSON

app_name = "job-hello_world:develop"

## 1. Deploy a job
Before deploying Applications and working with jobs, it is assumed that a system administrator has deployed an ADES. These are often called "venues" or "environments", and some examples may be dev, test, prod, etc. To run a job, you need an an Application to deployed as well. In this case we have a demo application deployed and referenced in the setup step 0 above.

With an ADES and a deployed Application ready to accept our job, we can submit a job along with any input data or parameters that are needed (as defined by a template job definition for a particular Application). In this example case, none are needed so `inputs` is blank. The response will provide a Job ID that we will store in a variable called `job_id` for use later.

In [ ]:
data = {
  "mode": "async",
  "response": "document",
  "inputs": [
  ],
  "outputs": [
    {
      "id": "output",
      "transmissionMode": "reference"
    }
  ]
}

try:

    # Store Job ID to use in future steps
    job_id = api.submit_job(app_name, data)

    # If the job submission is successful, print a success message along with the returned JOB-ID
    print("\nJob Submission Successful!\nJOB ID: {}\n".format(job_id))

except requests.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)

## 2. Get the job status
The code below will demonstrate how one can check the status of a job. The potential status responses are documented [here] _need a reference to process status_.

In this example, we will look up the status of the predfined application name from the setup step 0, and the job ID that was returned previously.

In [ ]:
try:
    
    job_status = api.get_job_status(app_name, job_id)
    
    # Print the job status
    print("\nStatus for job \"{}\": {}\n".format(job_id, job_status))
    
except requests.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)


## 3. Get job results
Now that we've monitored the status of a job, and verified that is has completed, we can retreive information about where the generated data is located. 

In this example, we will use the predfined application name, and the job ID that was returned previously.

In [ ]:
try:

    job_data = api.get_job_result(app_name, job_id)

    for data in job_data:
        print("ID: " + data['id'])
        print("Type: " + data['mimeType'])
        print("Location: " + data['href'])
        
except requests.exceptions.HTTPError as e:
    print(e.reason)

print("\nFull output data object:")
JSON(job_data)